### Visualizing robot senses using ROS and Bokeh library

As you can see, robots publish many topics that could be interesting to see. 
Visualizing state of a sensor is the first step to understand it.

We will start with visualizing the state of the turtlesim through some cool techniques.

This time we will use the local server -- your own computer will become **[rosmaster](http://wiki.ros.org/rosmaster)**. 

In [3]:
%env ROS_MASTER_URI=http://localhost:11311
        
import rospy
import roslaunch
import rosnode
import os

env: ROS_MASTER_URI=http://localhost:11311


We will start *roscore* on your own machine

In [4]:
import subprocess
roscore_process=subprocess.Popen(["roscore"])

### Launching additional nodes

While our script will become a ROS node actually we can even launch additional nodes from it, using *roslaunch*.
**[roslaunch](http://wiki.ros.org/roslaunch)** is a set of utlility functions to help start, run and stop ROS nodes. While we will not go into detail here about how ROS launch can be used, this is a simple use case -- launching a set of ROS nodes and keeping the references to a processes in a list.

In [ ]:

import roslaunch

launch = roslaunch.scriptapi.ROSLaunch()
launch.start()

node1 = roslaunch.core.Node("ipython_robot_prototyping",
                            "roomba_simulator.py", name="roomba_sim") # this starts a ROS Node from ipython_robot_prototyping 
node2= roslaunch.core.Node("turtlesim", "turtlesim_node", name="turtlesim_node") #this starts our turtle

node3 =roslaunch.core.Node("topic_tools", "throttle", args="messages turtle1/distance 10.0", name="distance_throttle") #here we define node with additional parameters

node4 = roslaunch.core.Node("topic_tools", "throttle", args="messages turtle1/pose 10", name="pose_throttle")


all_nodes=[node1,node2,node3,node4]



all_processes=[launch.launch(node) for node in all_nodes]

#we can launch the processes individually
#roomba_process= launch.launch(node1)

#turtle_process=launch.launch(node2)


#distance_throttle_process=launch.launch(node3)

#pose_throttle_process=launch.launch(node4)


In [ ]:
### we can close the ROS nodes from ipython too

for process in all_processes:
    process.stop()

    
# or individually    
#distance_throttle_process.stop()
#pose_throttle_process.stop()

#roomba_process.stop()
#turtle_process.stop()

With the fallowing commands we have started two ROS Nodes -- one is a familiar turtlebot simulator and the other is a simple control node for the turtle that turns turtle around when she is near the wall so that it does not crash.


What is also does is it calculates the distance of the turtle to the wall she is heading to.

Lets see all the turtle's topics

In [ ]:
turtle_topics= tuple((topic_name,topic_type) 
                     for topic_name, topic_type in rospy.get_published_topics()
                     if "turtle" in topic_name)

print(turtle_topics)

There are couples of interesting topics there, like distance or pose. How can we visualize them?

In the case of turtlesim it is easy to see what the distance or position are by just looking at it. That is nice but in the case of real robots, we are not only interested in the actual position and orientation but also on robots _sensor_ readings that are, **[robots sensors](http://wiki.ros.org/Sensors)**. The distance in /turtle1/distance is something calculated, but does it actually make sense? It is very important to have a way to visualize sensor data and luckily, there are many ways in ROS to do it:

 - The easiest way is to use** _rostopic echo_ **command. You just put, for example, rostopic echo /turtle1/distance to see the distance values. Try to do it now.
 
 - The second option is to use another ROS command -- **[rqt_plot](http://wiki.ros.org/rqt_plot)**.

In [ ]:
os.system("rqt_plot /turtle1/distance/range") #Have fun, and close this new window before next step.


The command is nice as it automatically scales a plot to fit the changing data. It also allows you to add different topics through the graphical user interface or by putting different topic names when starting the tool (i.e. rqt_prot /turtle1/distance/range /turtle1/pose/x ). There is also a settings button in the right top corner to further set things up

 - It would be nice though to see the things directly in the Ipython notebook, to be able to see how the things change when we play around. We will show you couple of ways to do that.

### Ipython widgets

The first way to see sensor data in Ipython, especially if there is a single changing value, is to use **slider** but as an output.

To do this we will change the slider's *value* parameter in Subscriber callback function.

First, let's create a slider. Because the range is a float, we will be using a float slider. 

In [ ]:
from ipywidgets import FloatSlider
from IPython.display import display
distance_slider = FloatSlider(
    value=7.5,
    min=0,
    max=10.0,
    step=0.1,
    description='The distance to the wall',
)

distance_slider

Now, let's create a subscriber with a callback function that changes value of the slider. 



In [ ]:
own_node=rospy.init_node("distance_reader")

In [ ]:
from sensor_msgs.msg import Range
def read_distance_show_on_slider(range_msg):
    ''' The callback function that reads the range message from the turtle and presents it on a slider'''

    distance_slider.value=range_msg.range


distance_subscriber= rospy.Subscriber('turtle1/distance_throttle',
                                      Range, 
                                      read_distance_show_on_slider)

In [ ]:
### this will stop updating the slider
distance_subscriber.unregister()

We can see that the slider created before runs on its own! Now it is much easier to understand what the robot "sees" when running around.

Notice that our callback function has a second argument, to have room for some universality. We pass value to this argument by putting it to *callback_args* parameter when defining the Subscriber object.

### Exercise:

There is also a control message from our "roomba simulator" (on topic /turtle1/cmd_vel) .

Try to read the control message to see how they synchronize. Create two new sliders one for *cmd_vel.linear.x* and the second for *cmd_vel_angular.z*.

Put the 3 sliders one below the other to see how these values are synchronized.

In [ ]:
linear_vel_slider = FloatSlider(
    value=0,
    min=-2,
    max=2,
    step=0.1,
    description='The forward speed',
)

angular_vel_slider = FloatSlider(
    value=0,
    min=-10,
    max=10,
    step=0.1,
    description='The angular speed',
)

In [ ]:
### define the callback function
from geometry_msgs.msg import Twist

def read_x_z_show_on_slider(twist_msg):
    '''this function reads twist_msg coming from cmd_vel topic
    and sends it to sliders'''
    try:
        linear_vel_slider.value=...
        angular_vel_slider.value=...
    except Exception as e:
        print("There was an error",e)
        cmd_vel_subscriber.unregister()
        
### define here the two subscribe objects

cmd_vel_subscriber = rospy.Subscriber('turtle1/cmd_vel', ..., ...)


*HINT:*

In [ ]:
import rostopic
import rosmsg
## code HINT
# to see what cmd_vel is and how it is structured use
print(" this is what /turtle1/cmd_vel is")
print(rostopic.get_info_text("/turtle1/cmd_vel"))

# this command shows that the type of the command is geometry_msgs/Twist

print("this is how Twist msg is constructed \n")
print(rosmsg.get_msg_text("geometry_msgs/Twist"))

In [ ]:
### we can display all the sliders here

display(distance_slider)
display(linear_vel_slider)
display(angular_vel_slider)

In [ ]:
# here we finish the topic subscription
cmd_vel_subscriber.unregister()

distance_subscriber.unregister()



### Using traitlets to follow the changes to our topic values

While the previous approach was ok, you can see that we were mixing the display of the data (slider) with some logic -- the things we do when reading the values. We can do it in a nicer way using objects. Our object will have _traits_ -- special elements that will we will know when they change.

In [ ]:
import traitlets

#from geometry_msgs.msg import Twist
from turtlesim.msg import Pose
from sensor_msgs.msg import Range
class Gather_Data(traitlets.HasTraits):
    '''A class that gathers information from two topics.
    
    Attrs:
        distance: a traitlet for the distance of the turtle from the wall
        lin_speed: a traitlet for the pose.linear_speed 
        ang_speed: a traitlet for the pose.angular_speed
        x: a traitlet for the x position of the turtle, pose.x
        y: a traitlet for the y position of the turtle, pose.y
        theta: a traitlet for the theta postion of the turtle, pose.theta
        position: a traitlet (touple) for the (x,y,theta)'''
        
    # here we define a 7 traits that we will fallow
    distance=traitlets.Float()
    lin_vel=traitlets.Float()
    ang_vel=traitlets.Float()
    x=traitlets.Float()
    y=traitlets.Float()
    theta=traitlets.Float()
    position=traitlets.Tuple()
    def __init__(self,turtle_name="turtle1"):
        '''we initialize subscribers here'''
        self.distance_subscriber= rospy.Subscriber(turtle_name+"/distance_throttle",
                                      Range, 
                                      self.read_distance)
        self.pose_subscriber = rospy.Subscriber(turtle_name+"/pose_throttle",Pose,self.read_pose)
    
    def unregister(self):
        '''a method for unregistering to all the topics the object is registered to'''
        self.distance_subscriber.unregister()
        self.cmd_vel_subscriber.unregister()
        
    def __del__(self):
        '''unregister whe deleted'''
        self.unregister()
        
    def read_distance(self,distance_msg):
        '''distance callback for topic distance'''
        self.distance=distance_msg.range # we put the value of range to a distance _traitlet_   
        
    def read_pose(self,pose):
        '''a callback function that reads all the poses and saves them as object attributes'''
        self.lin_vel= pose.linear_velocity
        self.ang_vel = pose.angular_velocity
        self.x=pose.x
        self.y=pose.y
        self.theta=pose.theta
        self.position=(pose.x,pose.y,pose.theta)

What we did is that we defined a class, that gathers all the data for us. This class "has Traits" that is, some elements are followed and we can use this to do interesting things, without clogging the class itself. But first, let's see what the class does alone.

In [ ]:
### We unregister first, if there is any object at all
try:
    data_gatherer.unregister()
except:
    pass
data_gatherer=Gather_Data()

In [ ]:
#Try to run it couple of times:

print("distance to the wall",data_gatherer.distance)
print("linear velocity of the turtle",data_gatherer.lin_vel)
print("angular velocity of the turtle",data_gatherer.ang_vel)

Now our object gathers the data "in the background" but we can do some interesting stuff with that.

For example, we can move our sliders by connecting them to the traitlets

In [ ]:
### we redefine the sliders in case you don't run all the steps one after another
from ipywidgets import FloatSlider
from IPython.display import display


linear_vel_slider = FloatSlider(
    value=0,
    min=-2,
    max=2,
    step=0.1,
    description='The forward speed',
)

angular_vel_slider = FloatSlider(
    value=0,
    min=-10,
    max=10,
    step=0.1,
    description='The angular speed',
)

distance_slider = FloatSlider(
    value=7.5,
    min=0,
    max=10.0,
    step=0.1,
    description='The distance to the wall',
)

We connect the sliders using a dynamic link (dlink) from traitlets library

In [ ]:
#### here we connect the sliders to our traitlets

from traitlets import dlink

dl1=dlink((data_gatherer,'distance'),(distance_slider,'value'))
dl2=dlink((data_gatherer,'lin_vel'),(linear_vel_slider,'value'))
dl3=dlink((data_gatherer,'ang_vel'),(angular_vel_slider,'value'))
      


In [ ]:
display(distance_slider)
display(linear_vel_slider)
display(angular_vel_slider)

In [ ]:
### we can unlink using the unlink method

dl1.unlink()
dl2.unlink()
dl3.unlink()

### Exercise:

The pose message that we are reading has also the position itself, x, y,theta.

Link these 3 traitlets to 3 more sliders to see them all!


In [ ]:
from ipywidgets import FloatSlider
from IPython.display import display
from math import pi

# put your code below


In [ ]:
#display sliders here


In [ ]:
#unlink


### Using bokeh to visualise the data on the graph

While for some data a simple moving slider is enough to realise what is going on, plots can also boost our prototyping effort.

In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.resources import Resources
output_notebook(resources=Resources()) #this is for offline work

from bokeh.models import ColumnDataSource, Slider, Select


First we will define a bokeh plot.

In [ ]:
x=[0]
y=[0]

source = ColumnDataSource(data=dict(x=x, y=y))

p = figure(title="turtle position", plot_height=300, plot_width=600,x_range=(0,10), y_range=(0,10))
#r = p.line(x, y, color="#2222aa", line_width=3)


pose_point=p.circle('x','y',source=source, size=10)
handle=show(p, notebook_handle=True)


Now let's plot a point on this graph, using a function.

In [ ]:
def plot_point(position):
    '''will plot the point delivered by position:
    
    Args:
        position: a touple (x,y,theta) with a position'''
    source.stream(dict(x=[position[0]], y=[position[1]]),rollover=1)
    
    push_notebook(handle=handle)

Now we can use this function, to make changes on our plot, notice that the point moves when you change the values in the touple.

In [ ]:
plot_point((1,1)) #we have changed the position of the point

Similarely to the sliders, we will connect the plot to our data_gatherer, using an _observer_

In [ ]:
def pass_the_position_to_plot(update):
    '''this just passes the element "new" from the dictionary
    to the plot_point function'''
    plot_point(update["new"])

In [ ]:
data_gatherer.observe(pass_the_position_to_plot,names=['position'])

In [ ]:
data_gatherer.unobserve_all()

### Bonus Exercise

You can also use subscriber callback directely to plot point on the graph. Try to update plot
directely from a ("/turtle1/pose" subscriber callback)


Bonus info: Actually one subscriber can have multiple callback functions so we can add a new callback to 
data_gatherer.pose_subscriber 






In [ ]:
# bonus example, how to add additional callbacks

def some_new_callback(pose_msg):
    #here for example you can use plot_point function
    pass

data_gatherer.pose_subscriber.impl.add_callback(some_new_callback,None)

In [ ]:
#Your code


### Online plotting of time series

We can also plot the incoming topic messages on a timeseries, the syntax is very similar

In [ ]:
import roslib

source_timeseries = ColumnDataSource(data=dict(t=[], distance=[]))

plot_timeseries = figure(title="distance plotting", plot_height=300, plot_width=600)
#r = p.line(x, y, color="#2222aa", line_width=3)


plot_timeseries.line(x='t', y='distance', alpha=0.2, line_width=3, color='navy', source=source_timeseries)
#plot_timeseries.oval(x='t', y='distance', width=0.1, height=0.1, color='navy', source=source_timeseries)
handle_timeseries=show(plot_timeseries, notebook_handle=True)

def update_plot_timeseries(update):
    '''this takes the update and streams it to a plot'''
    
    time_point=float(rospy.get_rostime().to_sec())
    source_timeseries.stream(dict(t=[time_point], distance=[update["new"]]),rollover=1000)
     
    push_notebook(handle=handle_timeseries)

In [ ]:
data_gatherer.observe(update_plot_timeseries,names=['distance'])

In [ ]:
data_gatherer.unobserve_all()

### Cleanup

Before closing the notebook, let's do some cleanup.

In [ ]:
try:
    for process in all_processes:
        process.stop()
    
except:
    pass
finally:
    print("roslaunch processes stopped")

try:
    data_gatherer.unregister()
except:
    pass
finally:
    print("data gatherer unregistered")
rospy.signal_shutdown("closing the excercise")


In [2]:
roscore_process.kill()

## Wrapup

What you have learned:

1. How to use Ipython widgets as outputs.
2. How to write an object that collects and distributes information from different topics.
3. How to link traitlets and ipython widgets.
4. How to create an automatically updating plot with bokeh.
5. How to plot a time series with data from ROS.


## Great!

Next step: **[Controlling_and_visualizing_the_ESP_bot.ipynb](Controlling_and_visualizing_the_ESP_bot.ipynb)**.